In [4]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib
!pip install tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.1.0%2Bcu118-cp311-cp311-win_amd64.whl (2722.7 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.0%2Bcu118-cp311-cp311-win_amd64.whl (5.0 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.1.0%2Bcu118-cp311-cp311-win_amd64.whl (3.9 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.0-py3-none-any.whl (2.0 MB)
     ---------------------------------------- 0.0/133.1 kB ? eta -:--:--
     -------------------------------------- 133.1/133.1 kB 3.8 MB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/fsspec-2023.4.0-py3-none-any.whl (153 kB)
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     -------- ----------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\gupta\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from timeit import default_timer as timer
from tqdm.auto import tqdm

C:\Users\gupta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class FashionMNISTModel2(nn.Module):
  '''
  Model architecture that replicates TinyVGG
  '''
  def __init__(self, in_shape:int , hidden_units:int , output_shape:int):
    super().__init__()

    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_shape,
                  hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(hidden_units,
                  hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
     )

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(hidden_units,
                  hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(hidden_units,
                  hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
     )

    self.ClassifierLayer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(hidden_units*49,
                  output_shape)
    )

  def forward(self , x):
    x = self.conv_block1(x)
    # print(x.shape)
    x = self.conv_block2(x)
    # print(x.shape)
    x = self.ClassifierLayer(x)
    return x

In [11]:
model_1 = FashionMNISTModel2(in_shape = 1,
                             hidden_units = 30,
                             output_shape = 10)

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_1.parameters(),
                              lr = 0.01)

def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct / len(y_pred)) * 100
  return acc



def print_train_time(start :float,
                     end: float ,
                     device : torch.device=None):
  timee = end - start
  print(f"The time taken is: {timee}")
  return timee


In [15]:
# import pandas as pd

dataset = pd.read_csv('train.csv')

# Training Data
X_train = dataset.iloc[:40000, 1:].values
y_train = dataset.iloc[:40000, 0].values

X_training1 = torch.from_numpy(X_train)
y_training = torch.from_numpy(y_train)
X_training = torch.reshape(X_training1 , (40000 ,1 , 28,28))

X_training = torch.divide(X_training,255)

# Testing Data

X_test = dataset.iloc[40000:, 1:].values
y_test = dataset.iloc[40000:, 0].values

X_testing1 = torch.from_numpy(X_test)
y_testing = torch.from_numpy(y_test)
X_testing = torch.reshape(X_testing1 , (2000 ,1 , 28,28))

X_testing = torch.divide(X_testing,255)

In [18]:
# import torch
ds = torch.utils.data.TensorDataset(X_training , y_training)
train_dl = torch.utils.data.DataLoader(ds, batch_size = 128)

ds1 = torch.utils.data.TensorDataset(X_testing , y_testing)
test_dl = torch.utils.data.DataLoader(ds1, batch_size = 128)

In [20]:
def training_model(model: torch.nn.Module,
                   dataloader: torch.utils.data.DataLoader,
                   loss_fn: torch.nn.Module,
                   optimizer: torch.optim.Optimizer,
                   accuracy_fn):


  train_loss , train_acc = 0,0

  for batch ,(X,y) in enumerate(dataloader):

    model.train()

    y_preds = model(X)

    loss = loss_fn(y_preds , y)
    train_loss += loss

    train_acc += accuracy_fn(y , y_preds.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  print(f"Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.4f}")



In [21]:
def testing_model(model: torch.nn.Module,
                  dataloader: torch.utils.data.DataLoader,
                  loss_fn: torch.nn.Module,
                  accuracy_fn):
  test_loss ,test_acc = 0 ,0

  model.eval()

  with torch.inference_mode():
    for X_test,y_test in dataloader:

      test_pred = model(X_test)

      test_loss += loss_fn(test_pred , y_test)
      test_acc += accuracy_fn(y_test , test_pred.argmax(dim=1))

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

In [23]:
torch.manual_seed(42)

epochs = 20 # Increase it to increase the accuracy of the model

time_start = timer()
for epoch in tqdm(range(epochs)):
  print(f"Epoch {epoch}\n -------------")
  training_model(model_1,
                 train_dl,
                 loss_fn,
                 optimizer,
                 accuracy_fn)
  testing_model(model_1,
                 test_dl,
                 loss_fn,
                 accuracy_fn)

time_end = timer()

time_taken = print_train_time(time_start , time_end)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0
 -------------
Training Loss: 0.0510 | Training Accuracy: 98.4600


  5%|▌         | 1/20 [00:19<06:05, 19.24s/it]

Test Loss: 0.0763 | Test Accuracy: 97.6562
Epoch 1
 -------------
Training Loss: 0.0492 | Training Accuracy: 98.5299


 10%|█         | 2/20 [00:38<05:51, 19.53s/it]

...

Test Loss: 0.0610 | Test Accuracy: 97.9492
Epoch 18
 -------------
Training Loss: 0.0295 | Training Accuracy: 99.1763


 95%|█████████▌| 19/20 [06:30<00:21, 21.04s/it]

Test Loss: 0.0604 | Test Accuracy: 97.9980
Epoch 19
 -------------
Training Loss: 0.0287 | Training Accuracy: 99.2163


100%|██████████| 20/20 [06:51<00:00, 20.58s/it]

Test Loss: 0.0598 | Test Accuracy: 97.9980
The time taken is: 411.50958399999945


In [25]:
dataset = pd.read_csv('test.csv')

# Training Data
X_pr = dataset.iloc[:, :].values

X_t1 = torch.from_numpy(X_pr)

X_t = torch.reshape(X_t1 , (28000 , 1 , 28,28))

X_t = torch.divide(X_t,255)


In [38]:
import csv
with open("result.csv" , "w+" , newline = "") as f:
  fwrite = csv.writer(f)

  fields = ["ImageId" , "Label"]

  fwrite.writerow(fields)

  pred = model_1(X_t).argmax(dim=1)

  for i in range(28000):
    fwrite.writerow([i+1 , pred[i].item()])